In [1]:
import base64
import os
import requests

#envファイルの読み込み
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
pip install opencv-python pyzbar

Note: you may need to restart the kernel to use updated packages.


In [4]:
# JANCODEAPI呼び出し
def jancode_app():
    product_json = {
        "codeNumber": "4901301446596",
        "codeType": "JAN",
        "itemName": "アタックZERO パーフェクトスティック 76本入り",
        "itemModel": "",
        "itemUrl": "https://www.jancodelookup.com/code/4901301446596/",
        "itemImageUrl": "https://image.jancodelookup.com/4901301446596.jpg",
        "brandName": "",
        "makerName": "花王株式会社",
        "makerNameKana": "カオウ",
        "ProductDetails": []
    }
    return product_json

In [5]:
# OPEN AI呼び出し関数
def openai_app():
    # プロンプト整形用テキスト
    # ここで地域名も入る（例：北海道）
    region = f"{st.session_state.todoufuken}"
    prompt_for_gpt = f"""
    以下の商品情報をもとに、アニメ風キャラクターをStable Diffusionで生成するための
    使える英語のテキストプロンプトを作成してください。
    
    キャラクターは商品「{product_json['itemName']}」を擬人化したもので、
    地域「{region}」のイメージを反映させます。
    
    キャラクターはデフォルメ強めのコミカルな「ちびキャラ（SDキャラ）」風で、
    レトロなカードバトルゲーム風イラストとして表現してください。
    太めのアウトライン、カラフルで派手な色彩、能力値や属性を感じさせる雰囲気を持たせてください。
    
    以下の要素を必ず英語プロンプトに含めてください：
    - **性格**：キャラクターの性格を具体的に描写（例：勇敢で元気、清潔感がある、戦闘好きなど）
    - **服装**：RPGキャラクター風の衣装。商品名を連想させるデザインを取り入れる
    - **小物・持ち物**：商品名をモチーフにしたデフォルメ武器・防具を装備
    - **姿勢**：戦闘ポーズ（カードバトルゲーム風の構え）
    - **背景**：地域の特徴（自然や建物など）を取り入れた、カードゲーム用イラスト風背景
    - **演出**：戦闘力や特殊技を発動しそうなエフェクト（光、オーラ、数字的な力を感じさせる演出）
    
    また、このキャラクターに合う短く覚えやすいキャラクター名も作成してください。
    キャラクター名はカタカナで8文字以内でお願いします。
    
    
    商品情報：
    - 商品名: {product_json['itemName']}
    - メーカー: {product_json['makerName']}
    - 商品画像URL: {product_json['itemImageUrl']}
    
    ※結果は以下の形式で出力してください：
    Prompt: <ここに英語のプロンプト>
    Character Name: <ここにキャラクター名>
    """
    
    
    # OpenAI APIでテキストプロンプト生成
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "あなたはアニメ風キャラクター化用プロンプト作成の専門家です。"},
            {"role": "user", "content": prompt_for_gpt}
        ],
        max_tokens=200
    )
    
    # 生成結果を取得
    generated_text = response.choices[0].message.content.strip()
    
    # ラベルで分割
    lines = generated_text.splitlines()
    sd_prompt = ""
    character_name = ""
    
    for line in lines:
        if line.lower().startswith("prompt:"):
            sd_prompt = line.split(":", 1)[1].strip()
        elif line.lower().startswith("character name:"):
            character_name = line.split(":", 1)[1].strip()
    
    return sd_prompt , character_name



In [7]:
# Stability_aiの関数
def stabilityai_app():
    stability_prompt = f"""{sd_prompt}"""
    
    
    response = requests.post(
        f"{stability_api_host}/v1/generation/{engine_id}/text-to-image",
        headers={
            "Content-Type": "application/json",
            "Accept": "application/json",
            "Authorization": f"Bearer {stability_api_key}"
        },
        json={
             "style_preset": "anime",
            "text_prompts": [
            {
                "text": f"{stability_prompt}"
            }
        ],
            "cfg_scale": 7,
            "height": 1024,
            "width": 1024,
            "samples": 1,
            "steps": 30,
        },
    )
    #画像保存。
    if response.status_code != 200:
        st.error(f"API エラーが発生しました。ステータスコード: {response.status_code}\n内容: {response.text}")
        return None
    else:
        data = response.json()
    
        image_base64 = data["artifacts"][0]["base64"]
        # 1. Base64 → バイナリデータに変換
        image_bytes = base64.b64decode(image_base64)
        
        # 2. BytesIO でメモリ上にファイルのようなオブジェクトを作る
        image_buffer = BytesIO(image_bytes)
        
        # 3. PIL で画像オブジェクトに変換
        image = Image.open(image_buffer)
        
        # 4. Streamlit に表示
        st.image(image, caption="生成画像", use_column_width=True)  

        return image  
          

In [ ]:
    # 3. Stability AIで画像生成
    response = requests.post(
        f"{stability_api_host}/v1/generation/{engine_id}/text-to-image",
        headers={
            "Content-Type": "application/json",
            "Accept": "application/json",
            "Authorization": f"Bearer {stability_api_key}"
        },
        json={
            "style_preset": "anime",
            "text_prompts": [{"text": sd_prompt}],
            "cfg_scale": 7,
            "height": 1024,
            "width": 1024,
            "samples": 1,
            "steps": 30,
        }
    )

    if response.status_code != 200:
        st.error(f"APIエラーが発生しました。ステータスコード: {response.status_code}\n内容: {response.text}")
        return None, None, None

    data = response.json()
    image_base64 = data["artifacts"][0]["base64"]
    image_bytes = base64.b64decode(image_base64)
    image = Image.open(BytesIO(image_bytes))
    st.image(image, caption=f"{character_name}", use_column_width=True)